In [1]:
using Random
using DataFrames
using CSV
using Statistics

struct Fighter
    name::String
    h::Int16
    f::Int16
    l::Int16
    t::Int16
end

const MXS = 10000
const MXV = 100
const MNV = 10

function random_fighter(h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    flag = true
    while flag
        h = h0 + rand(0:(MXV - h0), 1)[1]
        f = f0 + rand(0:(MXV - f0), 1)[1]
        l = l0 + rand(0:(MXV - l0), 1)[1]
        t = t0 + rand(0:(MXV - t0), 1)[1]
        mm = minimum([h, f])
        budget = h*f+h*l+f*t
        if budget <= MXS && budget + mm > MXS
            flag = false
            return Fighter("No Name", h, f, l, t)
        end
    end
end

function eval_battle(a::Fighter, b::Fighter)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    if a_finds && b_finds
        if a.t > b.t
            return 1
        end
        if a.t < b.t
            return -1
        end
        if a.t == b.t
            if a.l > b.l
                return 1
            end
            if a.l < b.l
                return -1
            end
            if a.l == b.l
                if a.h > b.h
                    return 1
                end
                if a.h < b.h
                    return -1
                end
                if a.h == b.h
                    return 0
                end
            end
        end
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        if a.l > b.l
            return 1
        end
        if a.l < b.l
            return -1
        end
        if a.l == b.l
            if a.t > b.t
                return 1
            end
            if a.t < b.t
                return -1
            end
            if a.t == b.t
                if a.f > b.f
                    return 1
                end
                if a.f < b.f
                    return -1
                end
                if a.f == b.f
                    return 0
                end
            end
        end
    end
end

function random_tournament_winner(c::Int)::Fighter
    if c==0
        return random_fighter()
    end
    a = random_tournament_winner(c-1)
    b = random_tournament_winner(c-1)
    res = eval_battle(a, b)
    if res == 1
        return a
    else
        return b
    end
end


tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)

function random_name_and_stat()::Fighter
    vp = [0, 0, 0, 0]
    nametype = rand(1:5)
    name = ""
    if nametype == 1 || nametype == 2
        nm = rand(names)
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        if nametype == 1
            name = string(nm, " the ", adj)
        end
        if nametype == 2
            name = string(adj, " ", nm)
        end
    end
    if nametype == 3 || nametype == 4
        nm = rand(names)
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        if nametype == 3
            name = string(nm, " the ", noun)
        end
        if nametype == 4
            name = string(noun, " ", nm)
        end
    end
    if nametype == 5
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        name = string(adj, " ", noun)
    end
    Fighter(name, vp[1], vp[2], vp[3], vp[4])
end

function rand_rename(a::Fighter, n_tries::Int = 10)::Fighter
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.t^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.t * b.t)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Fighter(best_b.name, a.h, a.f, a.l, a.t)
end

function eval_battle_list(a::Fighter, bs::Array{Fighter})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function pick_best(as::Array{Fighter}, bs::Array{Fighter})::Fighter
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Fighter}, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function fighters_to_df(as::Array{Fighter})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ts = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ts[ii] = as[ii].t    
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Fighter}, bs::Array{Fighter})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

eval_team_battle (generic function with 1 method)

In [2]:
function random_team(f::Function, team_size::Int)::Array{Fighter}
    team = Array{Fighter}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [3]:
library = Array{Fighter}(undef, 100000)
for i in 1:100000
    library[i] = rand_rename(random_fighter())
end

In [4]:
function evaluate_generator(f::Function, team_size::Int, n_times::Int = 1)::AbstractFloat
    scores = Array{AbstractFloat}(undef, n_times)
    for i0 in 1:n_times
        team = Array{Fighter}(undef, team_size)
        for i in 1:team_size
            team[i] = f()
        end
        bestscore = 999999
        exploiter = library[1]
        for i in 1:length(library)
            fighter = library[i]
            score = 0
            for j in 1:team_size
                score = score + eval_battle(team[j], fighter)
            end
            if score < bestscore
                bestscore = score
                exploiter = library[i]
            end
        end
        if n_times < 5
            println(exploiter)        
        end
        scores[i0] = bestscore/team_size
    end
    return mean(scores)
end

evaluate_generator (generic function with 2 methods)

In [89]:
# new random generation to more closely approximate Nash
function random_scheme_b(budget::Int = MXS, h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    opt = rand([1,1,1,1, 2,2, 3,3, 4,4,4, 5,5,5, 6,6, 7,7, 8,8, 9,9])
    flip = (rand() < 0.5)
    r1 = rand()
    r2 = rand()^(0.3)
    r3 = rand()
    #           1     2     3.    4.    5.    6.    7.    8.    9. 
    h_mins =  [03.5, 05.4, 06.8, 07.5, 08.6, 09.2, 09.6, 10.2, 10.5]
    h_maxs =  [05.4, 06.8, 07.2, 08.6, 09.2, 09.6, 10.2, 10.5, 11.0]
    hl_mins = [40.0, 20.0, 12.0, 10.0, 10.0, 11.0, 20.0, 20.0, 20.0]
    hl_maxs = [65.0, 72.0, 54.0, 30.0, 38.0, 50.0, 30.0, 30.0, 40.0]
    h_raw = h_mins[opt] + (h_maxs[opt] - h_mins[opt]) * r1
    hl_raw = hl_mins[opt] + (hl_maxs[opt] - hl_mins[opt]) * r2
    l_raw = hl_raw/h_raw
    b_r = 100 - hl_raw # budget remaining raw
    f_min = b_r/15 # find given 15 tickle
    f_min = max(1, (b_r - h_raw * f_min)/15)
    f_min = max(1, (b_r - h_raw * f_min)/15)
    f_max = min((b_r - 15)/h_raw, h_raw)
    f_max = min((b_r - f_max)/h_raw, h_raw)
    f_max = min((b_r - f_max)/h_raw, h_raw)
    f_raw = f_min + (f_max - f_min) * r2
    mult = sqrt(budget/100)
    h = round(mult * h_raw)
    f = round(mult * f_raw)
    l = round(mult * l_raw)
    t = trunc((budget - h*f - h*l)/f)
    budget_sofar = h*f + h*l + f*t
    #println((h_raw, f_raw, l_raw, hl_raw, b_r, f_min, f_max))
    #println((h_raw, f_raw, l_raw, h, f, l, t, opt, hl_raw, budget_sofar, r1, r2, r3))
    if (budget - budget_sofar) >= min(h, f)
        could_add_l = (budget - budget_sofar) >= h
        could_add_t = (budget - budget_sofar) >= f
        if could_add_l && !could_add_t
            l = l+trunc((budget - budget_sofar)/h)
        end
        if !could_add_l && could_add_t
            t = t+trunc((budget - budget_sofar)/f)
        end
        if could_add_l && could_add_t
            if fpart(lraw) > fpart(traw)
                l = l+trunc((budget - budget_sofar)/h)
            else
                t = t+trunc((budget - budget_sofar)/f)
            end
        end
    end
    h = convert(Int, h)
    f = convert(Int, f)
    l = convert(Int, l)
    t = convert(Int, t)
    if flip
        ff = Fighter(string("Scheme B-", opt), f, h, t, l)
    else
        ff = Fighter(string("Scheme B+", opt), h, f, l, t)
    end
    return ff
end

#evaluate_generator(random_scheme_a, 1000, 50)

random_scheme_b (generic function with 6 methods)

In [90]:
rt = random_team(() -> random_scheme_b(), 1000)
counters = random_team(() -> pick_best_rdmly(library, rt, 25), 100)
rt = random_team(() -> random_scheme_b(), 100)
for f in counters[1:5]
    v = eval_battle_list(f, rt)
    print(v)
    print(" ")
end
fighters_to_df(counters[1:5])


28 54 38 34 30 

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Mouse Oonagh,98,53,37,22
2,Kymoni the Ninja,73,65,36,40
3,Dancing Armadillo,74,62,44,34
4,Rhea the Ninja,72,65,44,33
5,Zyaira the Dynamic,94,40,57,22


In [91]:
rt = random_team(() -> random_scheme_b(), 100)
fighters_to_df(rt)[1:15, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Scheme B+7,97,53,25,45
2,Scheme B-5,55,88,57,23
3,Scheme B+5,89,62,38,17
4,Scheme B-7,63,97,15,30
5,Scheme B+6,95,51,36,34
6,Scheme B-1,48,49,27,129
7,Scheme B+9,107,51,37,11
8,Scheme B+1,43,35,122,92
9,Scheme B+6,92,49,53,12


In [7]:
budget = MXS
h0 = MNV
f0 = MNV
l0 = MNV
t0 = MNV

#opt = rand(1:9)
opt = 4
#flip = (rand() < 0.5)
flip = false
r1 = rand()
r2 = rand()^(0.3)
r3 = rand()
h_mins =  [04.0, 06.0, 07.0, 08.0, 10.0, 12.0, 12.5, 13.0, 13.5]
h_maxs =  [06.0, 07.0, 08.0, 10.0, 12.0, 12.5, 13.0, 13.5, 15.0]
hl_mins = [48.0, 50.0, 52.0, 11.0, 26.0, 20.0, 20.0, 20.0, 20.0]
hl_maxs = [70.0, 72.0, 74.0, 30.0, 78.0, 30.0, 30.0, 30.0, 40.0]
h_raw = h_mins[opt] + (h_maxs[opt] - h_mins[opt]) * r1
hl_raw = hl_mins[opt] + (hl_maxs[opt] - hl_mins[opt]) * r2
l_raw = hl_raw/h_raw
b_r = 100 - hl_raw # budget remaining raw
f_min = b_r/15 # find given 15 tickle
f_min = max(1, (b_r - h_raw * f_min)/15)
f_min = max(1, (b_r - h_raw * f_min)/15)
f_max = min((b_r - 15)/h_raw, h_raw)
f_max = min((b_r - f_max)/h_raw, h_raw)
f_max = min((b_r - f_max)/h_raw, h_raw)
f_raw = f_min + (f_max - f_min) * r2
mult = sqrt(budget/100)
h = round(mult * h_raw)
f = round(mult * f_raw)
l = round(mult * l_raw)
t = trunc((budget - h*f - h*l)/f)
budget_sofar = h*f + h*l + f*t
println((h_raw, f_raw, l_raw, hl_raw, b_r, f_min, f_max))
println((h_raw, f_raw, l_raw, h, f, l, t, opt, hl_raw, budget_sofar, r1, r2, r3))
if (budget - budget_sofar) >= min(h, f)
    could_add_l = (budget - budget_sofar) >= h
    could_add_t = (budget - budget_sofar) >= f
    if could_add_l && !could_add_t
        l = l+trunc((budget - budget_sofar)/h)
    end
    if !could_add_l && could_add_t
        t = t+trunc((budget - budget_sofar)/f)
    end
    if could_add_l && could_add_t
        if fpart(lraw) > fpart(traw)
            l = l+trunc((budget - budget_sofar)/h)
        else
            t = t+trunc((budget - budget_sofar)/f)
        end
    end
end
h = convert(Int, h)
f = convert(Int, f)
l = convert(Int, l)
t = convert(Int, t)
if flip
    ff = Fighter(string("Scheme B-", opt), f, h, t, l)
else
    ff = Fighter(string("Scheme B+", opt), h, f, l, t)
end
return ff

println((h, f, l, t, opt, budget_sofar, r1, r2, r3))
println(ff)

(9.825462703157513, 6.427759161751591, 3.001875108503988, 29.494811918142847, 70.50518808185716, 3.6382301653505977, 6.503955492334131)
(9.825462703157513, 6.427759161751591, 3.001875108503988, 98.0, 64.0, 30.0, 12.0, 4, 29.494811918142847, 9980.0, 0.9127313515787565, 0.9734111535864657, 0.4060806004371126)
(98, 64, 30, 12, 4, 9980.0, 0.9127313515787565, 0.9734111535864657, 0.4060806004371126)
Fighter("Scheme B+4", 98, 64, 30, 12)


## Testing Random Scheme B

In [8]:
f = random_scheme_b()
print(f)
f.h * f.f + f.h * f.l + f.f * f.t

Fighter("Scheme B+1", 47, 38, 120, 67)

9972

In [9]:
rt = random_team(() -> random_scheme_b(), 1000)
fighters_to_df(rt)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Scheme B+1,56,52,113,14
2,Scheme B+9,137,42,25,19
3,Scheme B-8,55,115,12,26
4,Scheme B-7,59,96,27,28
5,Scheme B+3,72,58,68,16
6,Scheme B+9,139,41,24,23
7,Scheme B-1,48,53,27,116
8,Scheme B+2,64,41,105,16
9,Scheme B+5,87,60,30,36


In [10]:
counters = random_team(() -> pick_best_rdmly(library, rt, 25), 100)
fighters_to_df(counters)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Monkey Emmaleigh,72,67,32,42
2,Sekani the Cricket,97,25,72,23
3,Maliek the Blimp,97,20,78,24
4,Asah the Blueberry,89,54,39,31
5,Ninja Eleana,68,75,33,35
6,Mouse Righteous,56,89,14,47
7,Aavyaan the Flirtatious,99,54,39,14
8,Wally the Engineer,10,88,32,100
9,Fish Sena,75,60,47,32


In [11]:
rt = random_team(() -> random_scheme_b(), 100)
0

0

In [12]:
for f in counters[1:20]
    v = eval_battle_list(f, rt)
    if v > 30
        println(f)
        println(v)
    end
end

Fighter("Monkey Emmaleigh", 72, 67, 32, 42)
58
Fighter("Sekani the Cricket", 97, 25, 72, 23)
42
Fighter("Maliek the Blimp", 97, 20, 78, 24)
46
Fighter("Asah the Blueberry", 89, 54, 39, 31)
36
Fighter("Ninja Eleana", 68, 75, 33, 35)
56
Fighter("Aavyaan the Flirtatious", 99, 54, 39, 14)
44
Fighter("Wally the Engineer", 10, 88, 32, 100)
42
Fighter("Fish Sena", 75, 60, 47, 32)
44
Fighter("Flirtatious Rascal", 93, 15, 85, 46)
40
Fighter("Goldfish Journeigh", 100, 12, 84, 33)
58
Fighter("Ancient Balloon", 49, 99, 24, 40)
42
Fighter("Fiery Doctor", 20, 95, 44, 76)
42
Fighter("Krysten the Sassy", 65, 68, 49, 35)
46
Fighter("Liev the Laptop", 66, 76, 36, 34)
56
Fighter("Israfil the Xylophone", 16, 98, 67, 75)
54
Fighter("Sheltering Graylen", 73, 63, 44, 34)
54
Fighter("Amonte the Ladder", 22, 97, 40, 72)
38
Fighter("Massive Doorman", 34, 93, 12, 69)
32
Fighter("Ramsi the Fish", 100, 32, 62, 18)
46


In [13]:
for f in rt[1:20]
    v = eval_battle_list(f, counters)
    if v < -80
        println(f)
        println(v)
    end
end

In [14]:
for f in rt
    v = eval_battle_list(f, counters)
    if v > 50
        println(f)
        println(v)
    end
end

In [15]:
evaluate_generator(random_scheme_b, 100, 20)

-0.7300000000000001

In [16]:
evaluate_generator(random_fighter, 100, 20)

-0.8600000000000001

In [17]:
evaluate_generator(() -> random_tournament_winner(1), 100, 20)

-0.8619999999999999

In [18]:
evaluate_generator(() -> random_tournament_winner(2), 100, 20)

-0.8370000000000001

In [19]:
evaluate_generator(() -> random_tournament_winner(3), 100, 20)

-0.808

In [20]:
evaluate_generator(() -> random_tournament_winner(4), 100, 20)

-0.7930000000000001

## Team battles

In [21]:
team_a = random_team(() -> rand_rename(random_scheme_b()), 100)
team_r0 = random_team(() -> rand_rename(random_tournament_winner(0)), 100)
team_r1 = random_team(() -> rand_rename(random_tournament_winner(1)), 100)
team_r2 = random_team(() -> rand_rename(random_tournament_winner(2)), 100)


100-element Vector{Fighter}:
 Fighter("Ingenious Yak", 67, 55, 69, 30)
 Fighter("Superficial Lion", 44, 71, 43, 70)
 Fighter("Snake Kathryne", 90, 78, 24, 10)
 Fighter("Legaci the Pig", 39, 83, 15, 74)
 Fighter("Cricket Emyrson", 66, 68, 67, 16)
 Fighter("Kasai the Urchin", 21, 93, 51, 75)
 Fighter("Kooky Cobbler", 66, 35, 67, 93)
 Fighter("Lion Malayjah", 56, 43, 62, 95)
 Fighter("Tiger Greysen", 52, 63, 34, 78)
 Fighter("Dalasia the Zany", 60, 94, 14, 37)
 Fighter("Wallet Angelino", 93, 32, 70, 16)
 Fighter("Yoyo Luxanna", 85, 37, 41, 91)
 Fighter("Yarizel the Duck", 63, 31, 93, 70)
 ⋮
 Fighter("Kiran the Ninja", 84, 78, 31, 10)
 Fighter("Aera the Pteranodon", 94, 29, 47, 98)
 Fighter("Itinerant Iridessa", 85, 48, 32, 66)
 Fighter("Jagger the Strawberry", 74, 33, 93, 20)
 Fighter("Young Commander", 86, 60, 49, 10)
 Fighter("Marmot Iyanuoluwa", 98, 38, 42, 56)
 Fighter("Marmot Peneloperose", 62, 31, 98, 64)
 Fighter("Boisterous Mantis", 54, 81, 27, 51)
 Fighter("Zyliah the Mushroom", 

In [22]:
eval_team_battle(team_a, counters)

-1

In [23]:
eval_team_battle(team_a, team_r0)

-1

In [24]:
eval_team_battle(team_a, team_r1)

-1

In [25]:
eval_team_battle(team_a, team_r2)

-1

In [26]:
eval_team_battle(team_r0, team_a)

1

In [27]:
eval_team_battle(team_r0, team_r1)

1

In [28]:
eval_team_battle(team_r0, team_r2)

-1

In [29]:
bs = team_a
as = team_r0
a_i = 1
b_i = 1
print("         ")
println(as[a_i])
print("         ")
println(bs[b_i])
while (a_i <= length(as)) && (b_i <= length(bs))
    res = eval_battle(as[a_i], bs[b_i])
    if res == 1
        println(string(as[a_i].name, " won against ", bs[b_i].name))
        b_i = b_i + 1
        print("         ")
        println(bs[b_i])
    else
        if res == 0
            println(string(as[a_i].name, " tied against ", bs[b_i].name))
            b_i = b_i + 1
            print("         ")
            println(bs[b_i])
        else
            println(string(as[a_i].name, " lost against ", bs[b_i].name))
        end
        a_i = a_i + 1
        print("         ")
        println(as[a_i])
    end
end
a_out = (a_i > length(as))
b_out = (b_i > length(as))
if a_out
    if b_out
        return 0
    else
        return -1
    end
else
    return 1
end


         Fighter("Umbrella Livienne", 95, 61, 31, 20)
         Fighter("Ninja Quintus", 89, 61, 35, 23)
Umbrella Livienne lost against Ninja Quintus
         Fighter("Crystal Warthog", 15, 85, 83, 88)
Crystal Warthog lost against Ninja Quintus
         Fighter("Neurotic Archer", 58, 66, 37, 61)
Neurotic Archer lost against Ninja Quintus
         Fighter("Gentle Butler", 72, 94, 21, 18)
Gentle Butler won against Ninja Quintus
         Fighter("Deen the Nerdy", 94, 66, 30, 14)
Gentle Butler won against Deen the Nerdy
         Fighter("Fish Matthan", 94, 59, 22, 40)
Gentle Butler won against Fish Matthan
         Fighter("Helicopter Ariabella", 55, 78, 24, 56)
Gentle Butler lost against Helicopter Ariabella
         Fighter("Shovel Torin", 43, 65, 57, 73)
Shovel Torin won against Helicopter Ariabella
         Fighter("Primordial Quilt", 136, 41, 29, 11)
Shovel Torin won against Primordial Quilt
         Fighter("Calm Sphere", 68, 85, 25, 29)
Shovel Torin lost against Calm Sphere
         

         Fighter("Kitten Gleyber", 48, 51, 98, 55)
Kitten Gleyber lost against Icosahedron Rayshaun
         Fighter("Itinerant Cat", 64, 58, 62, 40)
Itinerant Cat won against Icosahedron Rayshaun
         Fighter("Quail Chet", 82, 72, 32, 20)
Itinerant Cat lost against Quail Chet
         Fighter("Nice Cheetah", 67, 55, 20, 90)
Nice Cheetah lost against Quail Chet
         Fighter("Felicitas the Phoenix", 75, 51, 62, 29)
Felicitas the Phoenix won against Quail Chet
         Fighter("Fiery Mango", 95, 63, 30, 18)
Felicitas the Phoenix won against Fiery Mango
         Fighter("Hunter Cordae", 88, 61, 35, 25)
Felicitas the Phoenix won against Hunter Cordae
         Fighter("Furry Rogue", 98, 57, 27, 31)
Felicitas the Phoenix won against Furry Rogue
         Fighter("Sheltering Violin", 76, 57, 52, 30)
Felicitas the Phoenix won against Sheltering Violin
         Fighter("Wise Maison", 95, 64, 28, 19)
Felicitas the Phoenix won against Wise Maison
         Fighter("Unreliable Warrior", 47, 

LoadError: BoundsError: attempt to access 100-element Vector{Fighter} at index [101]

In [30]:
a_i, b_i

(94, 101)